# Thống kê âm tiết tiếng Việt từ file từ điển TXT (VDic_uni.txt)
Yêu cầu:

- (a) Số lượng âm tiết khác nhau trong từ điển
- (b) Số lượng âm tiết "khả dĩ" theo tổ hợp [Phụ_âm_đầu * Âm_Đệm * Âm_Chính * Âm_Cuối * Thanh_điệu]
- (c) So sánh (a) và (b)
- (d) Khai phá một số quy luật đồng xuất hiện (phonotactics) + gợi ý ngữ pháp/ngữ nghĩa từ nhãn

In [62]:
from pathlib import Path
import re
import csv

DICT_PATH = Path("VDic_uni.txt")

def load_dictionary(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        words = [line.strip() for line in f if line.strip()]
    return words

lines = load_dictionary(DICT_PATH)
print(f"Loaded {len(lines)} words from dictionary.")

Loaded 37464 words from dictionary.


In [63]:
# CÂU A: TÍNH SỐ LƯỢNG ÂM TIẾT KHÁC NHAU

# Trích headword từ mỗi dòng
def iter_headwords(lines):
    headwords: list[str] = []
    for i, line in enumerate(lines, 1):
        if not line:
            continue
        head = line.split("\t", 1)[0].strip()
        if head:
            headwords.append(head)
    return headwords

headwords = iter_headwords(lines)
print("Number of headwords", len(headwords))


Number of headwords 37464


In [64]:
# Phân tích âm tiết
# Tách headword thành âm tiết (theo khoảng trắng & dấu gạch nối), loại bỏ dấu câu
punct_strip = '.,;:"\'()[]{}!?«»“”‘’'
strip_table = str.maketrans("", "", punct_strip)

def syllabify(word: str) -> list[str]:
    parts = re.split(r"[\s-]+", word)
    result = []
    for part in parts:
        cleaned = part.translate(strip_table)
        if cleaned:
            result.append(cleaned.lower())
    return result

all_syllables = []
for h in headwords:
    all_syllables.extend(syllabify(h))
print("Number of syllables (with duplicates):", len(all_syllables))

Number of syllables (with duplicates): 75443


In [65]:
# Lấy số lượng âm tiết khác nhau
unique_syllables = set(all_syllables)

In [66]:
# CÂU B: TÍNH SỐ LƯỢNG ÂM TIẾT KHẢ DĨ
# [Phụ_âm_đầu (Onsets) * Âm_Đệm (Medials) * Âm_Chính (Nucleus) * Âm_Cuối (Codas) * Thanh_điệu (Tones)]

# Thanh điệu
TONE_CODE = {
    "a": (0,"a"), "á": (1,"a"), "à": (2,"a"), "ả": (3,"a"), "ã": (4,"a"), "ạ": (5,"a"),
    "ă": (0,"ă"), "ắ": (1,"ă"), "ằ": (2,"ă"), "ẳ": (3,"ă"), "ẵ": (4,"ă"), "ặ": (5,"ă"),
    "â": (0,"â"), "ấ": (1,"â"), "ầ": (2,"â"), "ẩ": (3,"â"), "ẫ": (4,"â"), "ậ": (5,"â"),
    "e": (0,"e"), "é": (1,"e"), "è": (2,"e"), "ẻ": (3,"e"), "ẽ": (4,"e"), "ẹ": (5,"e"),
    "ê": (0,"ê"), "ế": (1,"ê"), "ề": (2,"ê"), "ể": (3,"ê"), "ễ": (4,"ê"), "ệ": (5,"ê"),
    "i": (0,"i"), "í": (1,"i"), "ì": (2,"i"), "ỉ": (3,"i"), "ĩ": (4,"i"), "ị": (5,"i"),
    "y": (0,"y"), "ý": (1,"y"), "ỳ": (2,"y"), "ỷ": (3,"y"), "ỹ": (4,"y"), "ỵ": (5,"y"),
    "o": (0,"o"), "ó": (1,"o"), "ò": (2,"o"), "ỏ": (3,"o"), "õ": (4,"o"), "ọ": (5,"o"),
    "ô": (0,"ô"), "ố": (1,"ô"), "ồ": (2,"ô"), "ổ": (3,"ô"), "ỗ": (4,"ô"), "ộ": (5,"ô"),
    "ơ": (0,"ơ"), "ớ": (1,"ơ"), "ờ": (2,"ơ"), "ở": (3,"ơ"), "ỡ": (4,"ơ"), "ợ": (5,"ơ"),
    "u": (0,"u"), "ú": (1,"u"), "ù": (2,"u"), "ủ": (3,"u"), "ũ": (4,"u"), "ụ": (5,"u"),
    "ư": (0,"ư"), "ứ": (1,"ư"), "ừ": (2,"ư"), "ử": (3,"ư"), "ữ": (4,"ư"), "ự": (5,"ư"),
}
VOWELS_BASE = set(["a","ă","â","e","ê","i","y","o","ô","ơ","u","ư"])

# Phụ âm đầu
ONSETS = ["ngh","ng","gh","gi","kh","ph","qu","th","tr","ch","nh",
          "b","c","d","đ","g","h","k","l","m","n","p","q","r","s","t","v","x"]

# Phụ âm cuối
CODAS  = ["ch","nh","ng","c","m","n","p","t"]

In [67]:
def strip_tone(s: str) -> tuple[str, int]:
    out = []
    tone_index = 0
    for ch in s:
        if ch in TONE_CODE:
            t, base = TONE_CODE[ch]
            if tone_index == 0:
                tone_index = t
            out.append(base)
        else:
            out.append(ch)

    str_without_tone = "".join(out)
    return str_without_tone, tone_index

In [68]:
def split_syllable(syl: str) -> tuple[str, str, str, str, int] | None:
    s = syl.lower().strip()
    if not s:
        return None
    str_without_tone, tone_index = strip_tone(s)

    # Lấy phụ âm đầu
    onset = ""
    for o in ONSETS:
        if str_without_tone.startswith(o):
            onset = o
            break
    rest = str_without_tone[len(onset) :]
    if not rest:
        return None

    # Lấy phụ âm cuối/ âm cuối
    coda = ""
    for c in sorted(CODAS, key=len, reverse=True):
        if rest.endswith(c):
            coda = c
            rest = rest[: -len(c)]
            break
    if not rest:
        return None

    # Lấy âm đệm: 'o' hoặc 'u' đứng đầu phần vần (trừ 'qu' xem 'u' thuộc âm đầu)
    medial = ""
    if rest and rest[0] in ("o", "u") and not (onset == "qu" and rest[0] == "u"):
        medial = rest[0]
        rest = rest[1:]
    if not rest:
        return None

    # Lấy âm chính: nguyên âm gốc đầu tiên còn lại
    nucleus = next((ch for ch in rest if ch in VOWELS_BASE), "")
    if not nucleus:
        return None

    return onset, medial, nucleus, coda, tone_index


In [69]:
onset_set, medial_set, nucleus_set, coda_set, tone_set = (
    set(),
    set(),
    set(),
    set(),
    set(),
)

for s in all_syllables:
    parts = split_syllable(s)
    if parts:
        o, m, v, c, t = parts
        onset_set.add(o)
        medial_set.add(m)
        nucleus_set.add(v)
        coda_set.add(c)
        tone_set.add(t)

B = len(onset_set) * len(medial_set) * len(nucleus_set) * len(coda_set) * len(tone_set)

# Lưu danh sách âm tiết duy nhất
print("a) Số lượng âm tiết khác nhau trong từ điển: ", len(unique_syllables))
out_csv = Path("unique_syllables.csv")
with out_csv.open("w", encoding="utf-8", newline="") as f:
    w = csv.writer(f)
    for s in unique_syllables:
        w.writerow([s])
print("Lưu chi tiết tại:", out_csv)

print("===============================")
print("b) Số lượng âm tiết khả dĩ")
print("Số lượng phụ âm đầu:", len(onset_set))
print("Số lượng âm đệm:", len(medial_set))
print("Số lượng nguyên âm:", len(nucleus_set))
print("Số lượng âm cuối:", len(coda_set))
print("Số lượng thanh điệu:", len(tone_set))
print("Số lượng âm tiết khả dĩ theo tổ hợp [Phụ_âm_đầu * Âm_Đệm * Âm_Chính * Âm_Cuối * Thanh_điệu]", B)





a) Số lượng âm tiết khác nhau trong từ điển:  7994
Lưu chi tiết tại: unique_syllables.csv
b) Số lượng âm tiết khả dĩ
Số lượng phụ âm đầu: 28
Số lượng âm đệm: 3
Số lượng nguyên âm: 12
Số lượng âm cuối: 9
Số lượng thanh điệu: 6
Số lượng âm tiết khả dĩ theo tổ hợp [Phụ_âm_đầu * Âm_Đệm * Âm_Chính * Âm_Cuối * Thanh_điệu] 54432
